In [1]:
#Necessary Libraries
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import cv2 as cv
import random as rd

import torch#torch.utils.data.Dataset,torch.utils.data.DatLoader
import torch.nn as nn
import torchvision#transforms/models
import pickle

import PIL#Image, ImageDraw
import itertools
import subprocess
import multiprocessing
import os
import time
import copy
import zipfile
import csv
import shutil
from time import perf_counter
import torch
from sklearn.metrics import average_precision_score,roc_auc_score
from itertools import chain
#import utility

In [2]:
#All sort of Hyper Parameters
K = 0
K_FOLD = 5

IN_KERNEL = os.environ.get('KAGGLE_WORKING_DIR') is not None
TRAIN_BATCH_SIZE = 8
TEST_BATCH_SIZE = 8
NUM_WORKERS = 4
SEQUENCE_LENGTH = 18
MASKING_SIZE = 10
N_FRAME = 5
NUM_EPOCHS = 400
LR = 0.000001
PATH = str(K)+"-Fold-Files"
GPU = torch.cuda.is_available()
DEVICE = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")


DIRECTORIES = ["../input/crime-dataset/Assault","../input/crime-dataset/Shooting","../input/crime-dataset/Arson","../input/crime-dataset/Fighting","../input/crime-dataset/Robbery","../input/crime-dataset/Burglary","../input/crime-dataset/Explosion"]

#variable for storing classes and their paths

# PathsDict = {"Fighting":["../input/crime-dataset/Fighting","../input/crime-dataset/Assault"],
#            "Gun-Event":["../input/crime-dataset/Shooting"],#,"../input/crime-dataset/Robbery"
#              "Arson_Explosion": ["../input/crime-dataset/Arson","../input/crime-dataset/Explosion"]
#         }
# PathsDict = {"Non-Threat":["../input/crime-dataset/Normal_1","../input/crime-dataset/Normal_2","../input/crime-dataset/Normal_3"],
#              "Threat":["../input/crime-dataset/Assault","../input/crime-dataset/Fighting","../input/crime-dataset/Shooting","../input/crime-dataset/Arson","../input/crime-dataset/Explosion"]
#          }
PathsDict = {
            "Fighting":["../input/crime-dataset/Fighting"],
            "Gun-Event":["../input/crime-dataset/Shooting"],
            "Arson_Explosion": ["../input/crime-dataset/Arson","../input/crime-dataset/Explosion"],
            "Non-Threat":["../input/crime-dataset/Normal_3"]
}

In [3]:
#Displaying total files for each class
Files = {}
for key in PathsDict.keys():
    Files[key] = []
for Label in PathsDict:
    for DirPath in PathsDict[Label]:
        Temp = os.listdir(DirPath)
        for i in Temp:
            Files[Label].append((Label,os.path.join(DirPath,i)))
for key in PathsDict.keys():
    print(key+":",len(Files[key]))

Fighting: 61
Gun-Event: 55
Arson_Explosion: 110
Non-Threat: 90


In [4]:
#Helper function for Weighted K-Fold
def SplitDict(Dict,Splits):
    if np.sum(Splits)>1.0:
        print("Can't Split. Limit Exceeded!")
        return
    Splitted ={}
    for key in Dict:
        Splitted[key] = []
        rd.shuffle(Dict[key])
        Start = 0
        for i in range(len(Splits)):
            End = int(Start + Splits[i] * len(Dict[key]))
            if np.sum(Splits)==1.0 and i+1==len(Splits):
                Splitted[key].append(Dict[key][Start:])
            else:
                Splitted[key].append(Dict[key][Start:End])
            Start = End
    SplittedList = []
    for i in range(len(Splits)):
        SplittedList.append([])
        for key in Splitted:
            SplittedList[i]+=Splitted[key][i]
    return SplittedList

#Helper function for K-Fold
def SplitList(List,Splits):
    if np.sum(Splits)>1.0:
        print("Can't Split. Limit Exceeded!")
        return
    Start = 0
    SplittedList = []
    for i in range(len(Splits)):
        End = int(Start + Splits[i] * len(List))
        if np.sum(Splits)==1.0 and i+1==len(Splits):
            SplittedList.append(List[Start:])
        else:
            SplittedList.append(List[Start:End])
        Start = End
    return SplittedList

#Helper functions for accuracy and printing
def Accuracy(Y,Predictions):
    Correct=0
    for i in range(len(Predictions)):
        Index = Predictions[i].index(max(Predictions[i]))
        if Index==Y[i]:
            Correct+=1
    return Correct
def display(Type,epochNum,totalEpochs,Results):
    print( ''.join([Type," Epoch#",str(epochNum+1).zfill(3),'/',str(totalEpochs).zfill(3)]) ,end='' )
    for key in Results.keys():
        print(''.join([" ",key,str(Results[key])]),end='')
    print()
def zipdir(path, ziph):
    for root, dirs, files in os.walk(path):
        for file in files:
            ziph.write(os.path.join(root, file))

In [5]:
#Helper function for video segmentation

def calculate_first_mask(frames,skip=10):
    frame1=frames[0]
    gray1 = cv.cvtColor(frame1, cv.COLOR_BGR2GRAY)
    gray1 = cv.GaussianBlur(gray1, (3, 3), 0)
    length,width,color=frames[0].shape
    mask=np.zeros((length,width),dtype=np.uint8)
    dimension=frame1.shape[0]*frame1.shape[1]
    for i in range(1,skip):
        frame2=frames[i]
        gray2 = cv.cvtColor(frame2, cv.COLOR_BGR2GRAY)
        gray2 = cv.GaussianBlur(gray2, (3, 3), 0)
        deltaframe=cv.absdiff(gray1,gray2)
        threshold = cv.threshold(deltaframe, 20, i, cv.THRESH_BINARY)[1]
        threshold = cv.dilate(threshold,None,20)
        zero_frame=len(gray2[gray2==0])
        if zero_frame<dimension:
            mask=np.maximum(mask,threshold)
    return mask
def apply_mask(frames,skip=10):
    output=[]
    mask=calculate_first_mask(frames,skip)
    firstmask=np.zeros(frames[0].shape,dtype=np.uint8)
    firstmask[mask>0]=1
    tempo=frames[1]*firstmask
    plt.imshow(tempo)
    dimension=frames[0].shape[0]*frames[0].shape[1]
    for i in range(len(frames)):
        temp=np.zeros((frames[0].shape),np.uint8)
        temp[mask>0]=1
        output.append(frames[i]*temp)
        gray2 = cv.cvtColor(frames[i], cv.COLOR_BGR2GRAY)
        gray2 = cv.GaussianBlur(gray2, (3, 3), 0)
        zero_frame=len(np.where(gray2==0))
        mask[mask>0]-=1
        if i>0:
            deltaframe=cv.absdiff(gray1,gray2)
            threshold = cv.threshold(deltaframe, 20, skip, cv.THRESH_BINARY)[1]
            threshold = cv.dilate(threshold,None,20)
            zero_frame=len(gray2[gray2==0])
            if zero_frame<dimension:
                mask=np.maximum(mask,threshold)
        gray1=gray2
    return output

#Function for fetching frames from video
def getFrames(FilePath, VideoFPS, n_frame=10, totalimages=16):
    Images=[]
    VideoCap = cv.VideoCapture(FilePath)
    Start = 0
    End = int(VideoCap.get(cv.CAP_PROP_FRAME_COUNT)) - n_frame*(totalimages//3)
    if End<Start:
        End = Start
    point= rd.randint(Start, End)
    if point!=0:
        VideoCap.set(cv.CAP_PROP_POS_FRAMES, point)
    
    for i in range(totalimages*n_frame):
        hasFrames,image = VideoCap.read()
        if i==0:
            pad = np.zeros(image.shape,dtype=np.uint8)
        
        if i%n_frame==0 and hasFrames:
            Images.append(image)
            continue

        if len(Images)==totalimages or not hasFrames:
            for j in range(i,totalimages*n_frame):
                if j%n_frame==0:
                    Images.append(copy.deepcopy(pad))
            break
    return Images


In [6]:
#Class for fetching videos
class VideoLoader(torch.utils.data.Dataset):
    def __init__(self,VideosList,Labels, n_frame =10, sequence_length = 16, transform = None, train=False):
        self.VideosList = VideosList
        self.transform = transform
        self.Labels = Labels
        self.train = train
        if transform is None:
            self.transform = torchvision.transforms.ToTensor()
        self.n_frame = n_frame
        self.sequence_length = sequence_length

    def __len__(self):
        return len(self.VideosList)

    def __getitem__(self,idx):
        Label,Path = self.VideosList[idx]
        FPS = 30
        Frames = getFrames(Path, VideoFPS=FPS, n_frame = self.n_frame, totalimages=self.sequence_length)
        Frames=apply_mask(Frames,MASKING_SIZE)
        Length = len(Frames)
        if self.train:
            Flip = rd.randint(0,1)
            Rotation = rd.randint(-3,3)
            TrainTransform = torchvision.transforms.Compose([
                                                            torchvision.transforms.RandomHorizontalFlip(p=Flip),
                                                            torchvision.transforms.RandomAffine((Rotation,Rotation)),
                                                            self.transform
                                                            ])
            Frames = [TrainTransform(PIL.Image.fromarray(Frame)) for Frame in Frames]
        else:
            Frames = [self.transform(PIL.Image.fromarray(Frame)) for Frame in Frames]

        Frames = torch.stack(Frames)
        Frames = Frames.reshape(Length,3,264,264)
        return Frames,self.Labels.index(Label)

In [7]:
#Model Architecture
class vggCNNEncoder(nn.Module):
    def __init__(self, drop_p=0.3, num_classes=3):
        super(vggCNNEncoder, self).__init__()

        self.fc_hidden1, self.fc_hidden2 = fc_hidden1, fc_hidden2
        self.drop_p = drop_p
        self.num_classes = num_classes
        
        #VGG Backbone
        vgg16 = torchvision.models.vgg16(pretrained=True)
        modules = list(vgg16.children())[:-1]      # delete the last fc layer.
        self.vgg16 = nn.Sequential(*modules)
        self.fc1 = nn.Linear(vgg16.classifier[0].in_features, 512)
        self.bn1 = nn.BatchNorm1d(512, momentum=0.01)
        self.fc2 = nn.Linear(512, 512)
        self.bn2 = nn.BatchNorm1d(512, momentum=0.01)
        self.fc3 = nn.Linear(512, 300)

        self.LSTM = nn.LSTM(
                        input_size=300,
                        hidden_size=256,
                        num_layers=3,
                        batch_first=True)
        self.fc4 = nn.Linear(256, 128)
        self.fc5 = nn.Linear(128, self.num_classes)

    def forward(self, x_3d):
        cnn_embed_seq = []
        for t in range(x_3d.size(1)):
            with torch.no_grad():
                x = self.vgg16(x_3d[:, t, :, :, :])
                x = x.view(x.size(0), -1)

            x = self.bn1(self.fc1(x))
            x = nn.functional.relu(x)
            x = nn.functional.dropout(x, p=self.drop_p, training=self.training)
            x = self.bn2(self.fc2(x))
            x = nn.functional.relu(x)
            x = nn.functional.dropout(x, p=self.drop_p, training=self.training)
            x = self.fc3(x)
            cnn_embed_seq.append(x)
        cnn_embed_seq = torch.stack(cnn_embed_seq, dim=0).transpose_(0, 1)

        self.LSTM.flatten_parameters()
        RNN_out, (h_n, h_c) = self.LSTM(cnn_embed_seq, None)  
        x = self.fc4(RNN_out[:, -1, :])
        x = nn.functional.relu(x)
        x = nn.functional.dropout(x, p=self.drop_p, training=self.training)
        x = self.fc5(x)
        return x

In [8]:
#Class for EarlyStopping
class CheckPoint():
    def __init__(self,Patience,Path,earlyStopping=False):
        self.Patience = Patience
        self.Count = 0
        self.BestLoss = float('inf')
        self.BestEpoch = -1
        self.Path = Path
        self.earlyStopping=earlyStopping

    def check(self,epoch, loss, model, optimizer, Results):
        torch.save({"Model":model.state_dict(),"Optimizer":optimizer.state_dict()},self.Path+"/Model.pth")
        if loss>self.BestLoss:
            self.Count+=1
        else:
            self.Count = 0

        with open(self.Path+"/Results.txt", 'wb') as file:
            pickle.dump(Results,file)
        if loss<self.BestLoss:
            self.BestLoss = loss
            self.BestEpoch = epoch
            torch.save({"Model":model.state_dict(),"Optimizer":optimizer.state_dict()},self.Path+"/BestModel.pth")
#         zipf = zipfile.ZipFile(self.Path+".zip", 'w', zipfile.ZIP_DEFLATED)
#         zipdir(self.Path, zipf)
#         zipf.close()
        if self.earlyStopping:
            if self.Count==self.Patience:
                print("\nEarly Stopping!")
                print("Model didn't improved for",self.Patience,"epochs.")
                return False
            return True
        else:
            return True
    def load(self,Path,model,optimizer,results):
        Dict = torch.load(Path+"/Model.pth")
        model.load_state_dict(Dict["Model"])
        optimizer.load_state_dict(Dict["Optimizer"])
        pickle_file = open(Path+"/Results.txt", "rb")
        results=pickle.load(pickle_file)
        return model,optimizer,results

In [9]:
#Splitting Dataset into K-Fold
List = SplitDict(Files,[1.0/K_FOLD for i in range(K_FOLD)])
with open("./3Classifier-5-Fold-Split.txt", 'wb') as file:
    pickle.dump(List,file)
# pickle_file = open("../input/crime-dataset/4Classifier-5-Fold-Split.txt", "rb")
# List=pickle.load(pickle_file)

TrainFiles = []
TestFiles = []
for i in range(K_FOLD):
    if K==i:
        TestFiles = List[i]
    else:
        TrainFiles.append(List[i])
TrainFiles = list(chain.from_iterable(TrainFiles))
print("TOTAL FOLDS:",K_FOLD,"FOLD#",K)
print("Train Samples:",len(TrainFiles))
print("Test Samples:",len(TestFiles))

TOTAL FOLDS: 5 FOLD# 0
Train Samples: 253
Test Samples: 63


In [10]:
#Displaying Classes in the Train Test Split for each class
TrainCount = {}
TestCount = {}
for key in PathsDict.keys():
    TrainCount[key] = 0
    TestCount[key] = 0
for i in range(len(TrainFiles)):
    Name,Path = TrainFiles[i]
    TrainCount[Name]+=1

for i in range(len(TestFiles)):
    Name,Path = TestFiles[i]
    TestCount[Name]+=1
print(TrainCount)
print(TestCount)

{'Fighting': 49, 'Gun-Event': 44, 'Arson_Explosion': 88, 'Non-Threat': 72}
{'Fighting': 12, 'Gun-Event': 11, 'Arson_Explosion': 22, 'Non-Threat': 18}


In [11]:
weights = []
for i in TrainCount.keys():
    weights.append(0)
    weights[-1] = 1-TrainCount[i]/len(TrainFiles)
print(weights)

[0.8063241106719368, 0.8260869565217391, 0.6521739130434783, 0.7154150197628458]


In [12]:
#Creating Dataloaders and other necessary things i.e. Criterion, Optimizer, Model

#torchvision.transforms.Normalize(mean = [0.3422, 0.3477, 0.3535], std = [0.0200, 0.0203, 0.0203])
Transform = torchvision.transforms.Compose([
                                            torchvision.transforms.Resize((264,264)),
                                            torchvision.transforms.ToTensor(),
                                            ])

TrainSet = VideoLoader(TrainFiles,list(PathsDict.keys()),n_frame=N_FRAME,sequence_length=SEQUENCE_LENGTH,transform = Transform,train=True)
TestSet = VideoLoader(TestFiles,list(PathsDict.keys()),n_frame=N_FRAME,sequence_length=SEQUENCE_LENGTH,transform = Transform,train=False)

TrainLoader = torch.utils.data.DataLoader(TrainSet,batch_size=TRAIN_BATCH_SIZE,num_workers=NUM_WORKERS,shuffle=True)
TestLoader = torch.utils.data.DataLoader(TestSet,batch_size=TEST_BATCH_SIZE,num_workers=NUM_WORKERS,shuffle=False)

print("Train BatchSize:",len(TrainLoader),"Test BatchSize:",len(TestLoader))
Model=vggCNNEncoder(num_classes=len(TrainCount))
Criterion = torch.nn.CrossEntropyLoss(weight=torch.FloatTensor(weights))
Optimizer = torch.optim.Adam(Model.parameters(),lr=LR)

Checkpoint = CheckPoint(5,"./"+PATH, earlyStopping = False)
Model.to(DEVICE)
Criterion.to(DEVICE)
SOFTMAX = nn.Softmax(dim=1)

Train BatchSize: 32 Test BatchSize: 8


Downloading: "https://download.pytorch.org/models/vgg16-397923af.pth" to /root/.cache/torch/checkpoints/vgg16-397923af.pth


In [14]:
Results = []

In [15]:
os.mkdir("./"+PATH)

In [16]:
#Function for training model
def train(Model,Optimizer,Criterion,DataLoader):
    Model.train()
    loss_train = 0.0
    Correct = 0
    Total = 0
    Predictions = np.array([])
    TruePredictions = np.array([])
    for idx,data in enumerate(DataLoader):
        Images,Labels = data[0].to(DEVICE),data[1].to(DEVICE)
        Optimizer.zero_grad()
        Results = Model(Images)
        AfterSoftmax = SOFTMAX(Results).to("cpu").detach().numpy()
        loss = Criterion(Results,Labels)
        loss_train+=loss.item()
        loss.backward()
        Labels = Labels.to("cpu").detach().numpy()
        Predictions = np.append(Predictions, AfterSoftmax[:,1])
        TruePredictions = np.append(TruePredictions, Labels)

        Correct += Accuracy(Labels, AfterSoftmax.tolist())
        Total += len(Labels)
        Optimizer.step()

    #AveragePrecision = average_precision_score(TruePredictions,Predictions)
    #ROC_AUC_Score = roc_auc_score(TruePredictions,Predictions)
    Results = {
                "Loss:":loss_train/len(DataLoader),
                "Accuracy:":Correct/Total
                #"AvgPrecision:":AveragePrecision,
                #"ROC_AUC_Score:":ROC_AUC_Score
              }
    return (Model,Optimizer,Criterion,Results)

#Function for testing model
def test(Model,Criterion,DataLoader):

    Model.eval()
    loss_test = 0.0
    Correct = 0
    Total = 0
    Predictions = np.array([])
    TruePredictions = np.array([])
    for idx,data in enumerate(DataLoader):
        with torch.no_grad():
            Images,Labels = data[0].to(DEVICE),data[1].to(DEVICE)
            Results = Model(Images)
            AfterSoftmax = SOFTMAX(Results).to("cpu").numpy()
            loss = Criterion(Results,Labels)
            loss_test+=loss.item()

        Labels = Labels.to("cpu").numpy()
        Predictions = np.append(Predictions, AfterSoftmax[:,1])
        TruePredictions = np.append(TruePredictions, Labels)

        Correct += Accuracy(Labels, AfterSoftmax.tolist())
        Total += len(Labels)
    
    #AveragePrecision = average_precision_score(TruePredictions,Predictions)
    #ROC_AUC_Score = roc_auc_score(TruePredictions,Predictions)
    Results = {
                "Loss:":loss_test/len(DataLoader),
                "Accuracy:":Correct/Total
                #"AvgPrecision:":AveragePrecision,
                #"ROC_AUC_Score:":ROC_AUC_Score
              }
    return (Model,Criterion,Results)

In [17]:
#Code for Training and Testing
Start = len(Results)

for epoch in range(Start,NUM_EPOCHS):
    Model,Optimizer,Criterion,trainResults = train(Model,Optimizer,Criterion,TrainLoader)
    display("\nTrain",epoch,NUM_EPOCHS,trainResults)
    
    Model,Criterion,testResults = test(Model,Criterion,TestLoader)
    display("Test",epoch,NUM_EPOCHS,testResults)
    Results.append([trainResults,testResults])
    Checkpoint.check(epoch,testResults["Loss:"], Model, Optimizer, Results)


Train Epoch#001/400 Loss:1.3834159970283508 Accuracy:0.2845849802371542
Test Epoch#001/400 Loss:1.381566271185875 Accuracy:0.2857142857142857

Train Epoch#002/400 Loss:1.3832453452050686 Accuracy:0.28063241106719367
Test Epoch#002/400 Loss:1.381217285990715 Accuracy:0.2857142857142857

Train Epoch#003/400 Loss:1.382941897958517 Accuracy:0.2845849802371542
Test Epoch#003/400 Loss:1.3813364952802658 Accuracy:0.2857142857142857

Train Epoch#004/400 Loss:1.382415533065796 Accuracy:0.2885375494071146
Test Epoch#004/400 Loss:1.381351038813591 Accuracy:0.2857142857142857

Train Epoch#005/400 Loss:1.3827954977750778 Accuracy:0.2845849802371542
Test Epoch#005/400 Loss:1.3810304999351501 Accuracy:0.2857142857142857

Train Epoch#006/400 Loss:1.3835465461015701 Accuracy:0.2766798418972332
Test Epoch#006/400 Loss:1.38080295920372 Accuracy:0.2857142857142857

Train Epoch#007/400 Loss:1.383131381124258 Accuracy:0.2885375494071146
Test Epoch#007/400 Loss:1.3807331025600433 Accuracy:0.2857142857142857